# Symbolic minimum-error discrimination eigensystem for the two-box problem

## Context

In this notebook the eigensystem which leads to the optimal measurement for minimum-error discrimination (MD) for the two-box problem (2BP) is symbolically computed using the standard sympy methods

## Outputs

- Symbolically computed optimal measurement vectors for the 2BP
- Symbolically computed minimum expected error probability

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sympy.matrices import Matrix
from sympy import Rational, simplify, sqrt, Abs
from sympy.physics.quantum import TensorProduct

In [44]:
# functions

def gram_schmidt(V): # gram-schmidt orthonormalization
    orthogonal = []

    for i in range(len(V)):
        v = V[i].copy()
        
        for j in range(i):
            v = v - (v.T.conjugate() * orthogonal[j])[0] * orthogonal[j]
        
        v = v / sqrt((v.T.conjugate() * v)[0])
        orthogonal.append(v)
    
    return orthogonal

In [76]:
# simulation parameters

n = 4 # half-number of photons
N = 2*n # total number of photons
d = 2**N # hilbert space dimension
etaA = Rational(1,2) # prior probability of box A
etaB = 1-etaA # prior probability of box B

In [77]:
# load full density matrices

with open("../full_density_matrices/rhoA_N{:d}_sym.bin".format(N),"rb") as inf:
    rhoA = pickle.load(inf)

with open("../full_density_matrices/rhoB_N{:d}_sym.bin".format(N),"rb") as inf:
    rhoB = pickle.load(inf)

In [78]:
# compute optimal measurement matrix

lamda = etaB*rhoB - etaA*rhoA

In [79]:
# compute eigensystem

vecs = lamda.eigenvects()

In [80]:
# sort eigenvalues and eigenvectors

eigvals = np.array([], dtype='object')
eigvecs = []
counter = 0

for i in range(len(vecs)):
    #print("Eigenvalue:\t", vecs[i][0]) # uncomment to check eigenvalues
    #print("Multiplicity:\t", vecs[i][1] ) # uncomment to check multiplicity of eigenvalues
    
    for j in range(vecs[i][1]):
        eigvals = np.concatenate((eigvals, [vecs[i][0]]))
        vec = vecs[i][2][j]
        
        A = vec.T.conjugate() * vec
        vec = vec/sqrt(A[0])
        eigvecs.append(vec)
        
        #vec = np.array(vec).flatten().astype(np.float64)
        #print(vec)
        #eigvecs[:,counter] = vec
        
        counter += 1
    #print("\n")

inds = eigvals.argsort()
eigvals = eigvals[inds[::-1]] # sort in decreasing order
eigvecs = np.array(eigvecs)[inds[::-1]]

In [81]:
# find orthonormal bases for the positive, negative and null eigenspaces

orthonormalize = False # the gram-schmidt process can take too much time for n > 2 (N > 4) and not necessary

if orthonormalize:
    pos_vecs = eigvecs[eigvals > 0]
    pos_vecs = [Matrix(a) for a in pos_vecs]
    pos_vecs = gram_schmidt(pos_vecs)
    nul_vecs = eigvecs[eigvals == 0]
    nul_vecs = [Matrix(a) for a in nul_vecs]
    nul_vecs = gram_schmidt(nul_vecs)
    neg_vecs = eigvecs[eigvals < 0]
    neg_vecs = [Matrix(a) for a in neg_vecs]
    neg_vecs = gram_schmidt(neg_vecs)
    new_eigvecs = pos_vecs+nul_vecs+neg_vecs
else:
    new_eigvecs = eigvecs

In [82]:
# visualize orthonormalized eigenvectors

visualize = False # visualization can be too consuming for n > 2 (N > 4) and not necessary

if visualize:
    for v in new_eigvecs: display(simplify(v.T))

In [83]:
# save eigenvalues and eigenvectors in binary files

with open("../data/eigvals_md_2bp_sym_N{:d}.bin".format(N), "wb") as outf:
    pickle.dump(eigvals, outf)
    
with open("../data/eigvecs_md_2bp_sym_N{:d}.bin".format(N), "wb") as outf:
    pickle.dump(new_eigvecs, outf)

In [84]:
# save the minimum expected error probability

Perr = 0
for v in eigvals: Perr += Abs(v)
Perr = (1 - Perr)*Rational(1,2)
print(Perr)
np.savetxt("../data/perr_md_2bp_sym_N{:d}.txt".format(N), [Perr]) # save numerical (not symbolic) value

-sqrt(3)/14 - sqrt(55)/1120 + 1/4


In [85]:
Perr

-sqrt(3)/14 - sqrt(55)/1120 + 1/4